In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/2020.deeplearning/master/init.py
from init import init; init(force_download=False)

In [1]:
try:
    %tensorflow_version 2.x
    print ("Using TF2 in Google Colab")
except:
    pass
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard

from sklearn.datasets import *
from local.lib import mlutils
tf.__version__

'2.1.0'

#  `tf.function` 

## automatically converts pythonic code to a computational graph, using Tensors

In [2]:
def f(x):
    return x**2 + x*3
f(2)

10

In [3]:
@tf.function
def f(x):
    return x**2 + x*3
f(2)

<tf.Tensor: shape=(), dtype=int32, numpy=10>

and also works with a symbolic tensor

In [4]:
x = tf.Variable(3.)
f(x)

<tf.Tensor: shape=(), dtype=float32, numpy=18.0>

a `tf.function` is **traced** (converted to computation graph) the first time it is executed, then it is cached

In [5]:
@tf.function
def f47(x):
    print('Tracing!')
    tf.print('Executing')  
    return x**2 + x*47

In [6]:
f47(2)

Tracing!
Executing


<tf.Tensor: shape=(), dtype=int32, numpy=98>

In [7]:
f47(2)

Executing


<tf.Tensor: shape=(), dtype=int32, numpy=98>

observe that if the type changes, the function is **traced** again since a different computational graph must be created

In [8]:
f47(2.1)

Tracing!
Executing


<tf.Tensor: shape=(), dtype=float32, numpy=103.11>

observe the actual generated code by `tf.autograph`

In [9]:
print(tf.autograph.to_code(f47.python_function))


def tf__f47(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('f47', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
    print('Tracing!')
    ag__.converted_call(tf.print, ('Executing',), None, fscope)
    do_return = True
    retval_ = fscope.mark_return_value(x ** 2 + x * 47)
  do_return,
  return ag__.retval(retval_)



## performance of `tf.function`

In [10]:
x = tf.Variable(7.0)
@tf.function
def f1(x):
    
    def ff1(x):
        return x**2+x*3
    
    return ff1(x)

In [11]:
def f2(xval):
    
    @tf.function
    def ff2(x):
        return x**2+x*3

    return ff2(xval)

In [12]:
@tf.function
def f3(x):
    return x**2+x*3

In [13]:
@tf.function
def f3(x):
    return tf.pow(x,2)+tf.multiply(x,3)

def f4(x):
    return x**2 + x*3


In [14]:
f1(10.), f2(10.), f3(10.), f4(10.)

(<tf.Tensor: shape=(), dtype=float32, numpy=130.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=130.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=130.0>,
 130.0)

In [15]:
%timeit f1(10.)

90.9 µs ± 6.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [16]:
%timeit f2(10.)

3.11 ms ± 169 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%timeit f3(10.)

152 µs ± 28.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [18]:
%timeit f3.python_function(10.)

59.2 µs ± 2.65 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [19]:
%timeit f4(10.)

208 ns ± 20.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Underlying `concrete` functions are actual TF graphs with no polymorphism, tied to specific input types

`tf.function` maps python polymorphism to a set of different underlying concrete functions

In [20]:
@tf.function
def f(x):
    return x+x

In [21]:
f(10), f(10.), f("a")

(<tf.Tensor: shape=(), dtype=int32, numpy=20>,
 <tf.Tensor: shape=(), dtype=int32, numpy=20>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'aa'>)

observe different hash codes for each concrete function

In [22]:
fs = f.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string))
fs, fs(tf.constant("aa"))

(<tensorflow.python.eager.function.ConcreteFunction at 0x7f164c415ed0>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'aaaa'>)

In [23]:
fi = f.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.int32))
fi, fi(tf.constant(1))

(<tensorflow.python.eager.function.ConcreteFunction at 0x7f164c3ff1d0>,
 <tf.Tensor: shape=(), dtype=int32, numpy=2>)

In [24]:
ff = f.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.float32))
ff, ff(tf.constant(1.))

(<tensorflow.python.eager.function.ConcreteFunction at 0x7f164c45e9d0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.0>)

## `tf.function` with `keras` layers

In [25]:
import numpy as np
np.random.seed(0)
data = np.random.randn(3, 2)
data

array([[ 1.76405235,  0.40015721],
       [ 0.97873798,  2.2408932 ],
       [ 1.86755799, -0.97727788]])

In [26]:
inputer = tf.keras.layers.InputLayer(input_shape=(2))
denser1 = tf.keras.layers.Dense(4, activation='relu')
denser2 = tf.keras.layers.Dense(1, activation='sigmoid')

observe that, in eager mode, layers graphs are created as their code is being executed

In [27]:
def model_1(data):
    x = inputer(data)
    x = denser1(x)
    print('After the first layer:', x)
    out = denser2(x)
    print('After the second layer:', out)
    return out

print('Model output:\n', model_1(data))
print("--")
print('Model output:\n', model_1(data))

After the first layer: tf.Tensor(
[[0.         0.         0.72397983 0.        ]
 [0.         0.         1.0945917  0.        ]
 [0.02641147 0.         0.28564313 0.        ]], shape=(3, 4), dtype=float32)
After the second layer: tf.Tensor(
[[0.63380146]
 [0.69622225]
 [0.5481096 ]], shape=(3, 1), dtype=float32)
Model output:
 tf.Tensor(
[[0.63380146]
 [0.69622225]
 [0.5481096 ]], shape=(3, 1), dtype=float32)
--
After the first layer: tf.Tensor(
[[0.         0.         0.72397983 0.        ]
 [0.         0.         1.0945917  0.        ]
 [0.02641147 0.         0.28564313 0.        ]], shape=(3, 4), dtype=float32)
After the second layer: tf.Tensor(
[[0.63380146]
 [0.69622225]
 [0.5481096 ]], shape=(3, 1), dtype=float32)
Model output:
 tf.Tensor(
[[0.63380146]
 [0.69622225]
 [0.5481096 ]], shape=(3, 1), dtype=float32)


however, with `tf.function`, **FIRST** the function is **traced** resulting in a computational graph, which is what is **THEN** used in subsequent calls

In [28]:
@tf.function
def model_2(data):
    x = inputer(data)
    x = denser1(x)
    print('After the first layer:', x)
    out = denser2(x)
    print('After the second layer:', out)
    return out


print('Model\'s output:', model_2(data))
print('--')
print('Model\'s output:', model_2(data))

After the first layer: Tensor("dense/Relu:0", shape=(3, 4), dtype=float32)
After the second layer: Tensor("dense_1/Sigmoid:0", shape=(3, 1), dtype=float32)
Model's output: tf.Tensor(
[[0.63380146]
 [0.69622225]
 [0.5481096 ]], shape=(3, 1), dtype=float32)
--
Model's output: tf.Tensor(
[[0.63380146]
 [0.69622225]
 [0.5481096 ]], shape=(3, 1), dtype=float32)


`tf.function` usually requires less compute time, since in eager mode, everytime the function is called the graph is created

In [29]:
def model_1(data):
    x = inputer(data)
    x = denser1(x)
    out = denser2(x)
    return out

@tf.function
def model_2(data):
    x = inputer(data)
    x = denser1(x)
    out = denser2(x)
    return out

In [30]:
%timeit model_1(data)

622 µs ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [31]:
%timeit model_2(data)

225 µs ± 29.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### including graphs in upstream functions. 

observe how we compute the gradient of a computational graph:

- with `model_1` the graph is generated eagerly each time the function is called
- with `model_2` the graph is only generated in the first call

In [32]:
def g1(data):
    with tf.GradientTape() as t:
        y = model_1(data)

    return t.gradient(y, denser1.variables)

def g2(data):
    with tf.GradientTape() as t:
        y = model_2(data)

    return t.gradient(y, denser1.variables)

g2(data), g1(data)



([<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
  array([[-0.40976173,  0.        ,  0.81755775,  0.        ],
         [ 0.21442497,  0.        ,  0.24607055,  0.        ]],
        dtype=float32)>,
  <tf.Tensor: shape=(4,), dtype=float32, numpy=array([-0.21941043,  0.        ,  0.52378285,  0.        ], dtype=float32)>],
 [<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
  array([[-0.40976173,  0.        ,  0.81755775,  0.        ],
         [ 0.21442497,  0.        ,  0.24607055,  0.        ]],
        dtype=float32)>,
  <tf.Tensor: shape=(4,), dtype=float32, numpy=array([-0.21941043,  0.        ,  0.52378285,  0.        ], dtype=float32)>])

In [33]:
%timeit g1(data)

1.25 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [34]:
%timeit g2(data)

676 µs ± 65.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


however, even in `g2` the gradient graph is still computed eagerly.

if we wrap either function, now everything is a cached computational graph.

In [35]:
fg1 = tf.function(g1)
%timeit fg1(data)

198 µs ± 6.95 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [36]:
fg2 = tf.function(g2)
%timeit fg2(data)

226 µs ± 44.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
